In [ ]:
# if you have google drive permission then use this

from google.colab import drive
drive.mount('/gdrive')
!ln -s   /gdrive/MyDrive/colab/Credit_Card/data/ /content/data

In [ ]:
!pip3 install gdown

!mkdir -p ./data/
import gdown


url = 'https://drive.google.com/uc?id=%s'%("1NZ8jMoqcCDpydHxWm6j415HwVS093l4U")
output = './data/X_sample.csv'
gdown.download(url, output, quiet=False)


url = 'https://drive.google.com/uc?id=%s'%("18w2LpyPbydD9qoVQ33GZI2LOjyRY2LaI")
output = './data/y_sample.csv'
gdown.download(url, output, quiet=False)

url = 'https://drive.google.com/uc?id=%s'%("1-0TD5rSfzG3frbgT4fseGBn-Dn5OpGKy")
output = './data/X_test.csv'
gdown.download(url, output, quiet=False)

url = 'https://drive.google.com/uc?id=%s'%("1-1p1dHQVV2JnGP4eMfzJsQwErum_wuMG")
output = './data/y_test.csv'
gdown.download(url, output, quiet=False)

Downloading...
From: https://drive.google.com/uc?id=1NZ8jMoqcCDpydHxWm6j415HwVS093l4U
To: /content/data/X_sample.csv
48.9MB [00:00, 224MB/s]
Downloading...
From: https://drive.google.com/uc?id=18w2LpyPbydD9qoVQ33GZI2LOjyRY2LaI
To: /content/data/y_sample.csv
100%|██████████| 288k/288k [00:00<00:00, 57.8MB/s]
Downloading...
From: https://drive.google.com/uc?id=1-0TD5rSfzG3frbgT4fseGBn-Dn5OpGKy
To: /content/data/X_test.csv
609MB [00:02, 223MB/s]
Downloading...
From: https://drive.google.com/uc?id=1-1p1dHQVV2JnGP4eMfzJsQwErum_wuMG
To: /content/data/y_test.csv
3.59MB [00:00, 110MB/s]


'./data/y_test.csv'

In [ ]:
!pip3  install -U mlxtend

import pandas as pd
import matplotlib.pyplot as plt
import torch, torch.nn as nn, torch.nn.functional as F
from torch.autograd import Variable
import torch.utils.data as data_utils

     |████████████████████████████████| 1.4MB 7.4MB/s 
  Found existing installation: mlxtend 0.14.0
    Uninstalling mlxtend-0.14.0:
      Successfully uninstalled mlxtend-0.14.0


In [ ]:
from sklearn.metrics import roc_auc_score, confusion_matrix, precision_score, recall_score, f1_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score , average_precision_score
from sklearn.metrics import f1_score, confusion_matrix, precision_recall_curve, roc_curve ,auc , log_loss ,  classification_report

In [ ]:
df_X = pd.read_csv("./data/X_sample.csv")
df_X = df_X.set_index("txkey")

In [ ]:
df_y = pd.read_csv("./data/y_sample.csv")
df_y = df_y.set_index("txkey")

In [ ]:
import lightgbm as lgb


In [ ]:
from sklearn.model_selection import train_test_split
df_X_train, df_X_val = train_test_split(df_X, test_size=0.25, random_state=42)


In [ ]:
df_y_train = df_y.loc[df_X_train.index]
df_y_val = df_y.loc[df_X_val.index]

In [ ]:
class FraudNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(367, 16)
        self.fc2 = nn.Linear(16, 18)
        self.fc3 = nn.Linear(18, 20)
        self.fc4 = nn.Linear(20, 24)
        self.fc5 = nn.Linear(24, 1)
        self.bn1 = nn.BatchNorm1d(18)
        self.bn2 = nn.BatchNorm1d(24)
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.bn1(x)
        x = F.dropout(x, p=0.3)
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = self.bn2(x)
        x = F.sigmoid(self.fc5(x))
        return x

In [ ]:
net = FraudNet().cuda()
loss_fn = nn.BCELoss()
optimizer = torch.optim.Adam(net.parameters(), lr=1e-3)


training_epochs = 100
minibatch_size = 64

train = data_utils.TensorDataset(torch.from_numpy(df_X_train.values).float(), torch.from_numpy(df_y_train.values).float())
train_loader = data_utils.DataLoader(train, batch_size=minibatch_size, shuffle=True)

val = data_utils.TensorDataset(torch.from_numpy(df_X_val.values).float(), torch.from_numpy(df_y_val.values).float())
val_loader = data_utils.DataLoader(val, batch_size=minibatch_size, shuffle=True)

In [ ]:
for i in range(training_epochs):
    net.train()
    for b, data in enumerate(train_loader, 0):
        inputs, labels = data
        y_pred = net(inputs.cuda())
        loss = loss_fn(y_pred, labels.cuda())

        if b % 100:
            print('Epochs: {}, batch: {} loss: {}'.format(i, b, loss))
        #reset gradients
        optimizer.zero_grad()

        # backward pass
        loss.backward()

        # update weights
        optimizer.step()
    net.eval()     # Optional when not using Model Specific layer
    for b, data in enumerate(train_loader, 0):
        inputs, labels = data
        y_pred = net(inputs.cuda())
        loss = loss_fn(y_pred, labels.cuda())

        if b % 100:
            print('Epochs: {}, batch: {} loss: {}'.format(i, b, loss))

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1709: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


串流輸出內容已截斷至最後 5000 行。
Epochs: 92, batch: 329 loss: 0.0960298404097557
Epochs: 92, batch: 330 loss: 0.12502092123031616
Epochs: 92, batch: 331 loss: 0.05947546288371086
Epochs: 92, batch: 332 loss: 0.09243568778038025
Epochs: 92, batch: 333 loss: 0.14318878948688507
Epochs: 92, batch: 334 loss: 0.11521896719932556
Epochs: 92, batch: 335 loss: 0.10843313485383987
Epochs: 92, batch: 336 loss: 0.15774807333946228
Epochs: 92, batch: 337 loss: 0.19337040185928345
Epochs: 92, batch: 338 loss: 0.07710376381874084
Epochs: 92, batch: 339 loss: 0.11568503081798553
Epochs: 92, batch: 340 loss: 0.04466857388615608
Epochs: 92, batch: 341 loss: 0.06719991564750671
Epochs: 92, batch: 342 loss: 0.21048420667648315
Epochs: 92, batch: 343 loss: 0.13139638304710388
Epochs: 92, batch: 344 loss: 0.1016920655965805
Epochs: 92, batch: 345 loss: 0.15208888053894043
Epochs: 92, batch: 346 loss: 0.05821620672941208
Epochs: 92, batch: 347 loss: 0.15355363488197327
Epochs: 92, batch: 348 loss: 0.09142349660396576
E

In [ ]:
# create dataset for lightgbm
lgb_train = lgb.Dataset(df_X_train, df_y_train)
lgb_eval = lgb.Dataset(df_X_val, df_y_val, reference=lgb_train)

In [ ]:
# specify your configurations as a dict
params={}
params['learning_rate']=0.03
params['boosting_type']='gbdt' #GradientBoostingDecisionTree
params['objective']='binary' #Binary target feature
params['metric']='binary_logloss' #metric for binary classification
params['max_depth']=10

print('Starting training...')

gbm = lgb.train(params,
                lgb_train,
                num_boost_round=20,
                valid_sets=lgb_eval,
                early_stopping_rounds=20)



# Validation with downsampleing data

In [ ]:
print('Starting predicting...')
# predict
y_pred = gbm.predict(df_X_val, num_iteration=gbm.best_iteration)
# eval
print('auc:', roc_auc_score(df_y_val, y_pred))

In [ ]:
confusion_matrix(df_y_val, (y_pred > 0.5).astype(int))

In [ ]:
tn, fp, fn, tp  = confusion_matrix(df_y_val, (y_pred > 0.5).astype(int)).ravel()

In [ ]:
tn, fp, fn, tp

# Testing : verify with real distribution data

In [ ]:
X_test = pd.read_csv("./data/X_test.csv")
X_test = X_test.set_index("txkey")
y_test = pd.read_csv("./data/y_test.csv")
y_test = y_test.set_index("txkey")

In [ ]:
y_pred = net(torch.from_numpy(X_test.values).float().cuda()).detach().cpu().numpy()
y_test = y_test.values

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1709: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


In [ ]:

y_pred = gbm.predict(X_test, num_iteration=gbm.best_iteration)
y_test.mean()
print('auc:', roc_auc_score(y_test, y_pred))

In [ ]:
cm = confusion_matrix(y_test, (y_pred > 0.5).astype(int), labels=[0,1])
cm

array([[339736,  35639],
       [   352,   4720]])

In [ ]:
th = .6
print("precision_score", precision_score(y_test,(y_pred > th).astype(int)))
print("recall_score",    recall_score(y_test,(y_pred > th).astype(int)))
print("f1_score",        f1_score(y_test,(y_pred > th).astype(int)))

precision_score 0.12348354698030924
recall_score 0.9211356466876972
f1_score 0.21777332370009556


In [ ]:
#LGB
precision, recall, thresholds = \
    precision_recall_curve(y_test,y_pred)
average_precision = \
    average_precision_score(y_test,y_pred)

plt.step(recall, precision, color='k', alpha=0.7, where='post')
plt.fill_between(recall, precision, step='post', alpha=0.3, color='k')

plt.xlabel('Recall')
plt.ylabel('Precision')
plt.ylim([0.0, 1.05])
plt.xlim([0.0, 1.0])

plt.title('Precision-Recall curve: Average Precision = {0:0.2f}'.format(
          average_precision))

fpr, tpr, thresholds = \
    roc_curve(y_test,y_pred)
areaUnderROC = auc(fpr, tpr)

plt.figure()
plt.plot(fpr, tpr, color='r', lw=2, label='ROC curve')
plt.plot([0, 1], [0, 1], color='k', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic: \
Area under the curve = {0:0.2f}'.format(areaUnderROC))
plt.legend(loc="lower right")
plt.show()

In [ ]:
fnscore =  [ (fname,socre) for fname,socre in zip(gbm.feature_name(),gbm.feature_importance())]

In [ ]:
top_feature = sorted(fnscore, key=lambda tup: tup[1], reverse=True)[:25]

# Bias and Variance ，[參考資料1](https://machinelearningmastery.com/gentle-introduction-to-the-bias-variance-trade-off-in-machine-learning/), [參考資料二]
* 一般來模型的 Bias 和 Variance 會呈現互斥的現像
* 通常來說 Bias 越低，代表可能產生 Overfitting ，而 Overfitting 通常代表 Vairance 越高，其意義代表對於 Input 資料的敏感到越高。
## How to Caculate the Model's Bias
* Bias 的定義: Bias 是模型的預測(產出)的均值與實際值的差異(Bias is the difference between the mean of these estimates and the actual value.)
* Model Bias Formula : $$ {1 \over n}\sum\limits_{k=1}^n \{{\hat{y}-y_i}\}^2 $$
## How to Caculate the Model's Variance
* Variance 定義：這與實際(y label)的 value 沒關係，而是這個組模型的穩定度，在不同 sub training set 所產出相對應的 model 對於 同一組 testing data 所預測(產出)的值的 variance .
* Variance Defintion: Variance is the amount that the estimate of the target function will change if different training data was used.
*  Model Variance Formula:$$ {1 \over N}\sum\limits_{n=1}^N {1 \over L}\sum\limits_{l=1}^l \{{ y^l(x_n) -  \bar{y}(x_n) }\}^2 $$
* N 為 testing data 的數(筆)量，L 將 training data 折成多少個 sub trainin set (N is the number of rows that is in testing set . L is the number of subset that is splited from all training set. )

## 使用不同的模型來看 Bias 與 Variance 的變化

In [ ]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
lr.fit(df_X_train,df_y_train)
y_pred = lr.predict(X_test,)
print('auc:', roc_auc_score(y_test, y_pred))

In [ ]:
df_X_train = df_X_train.reset_index(drop=True)
df_y_train = df_y_train.reset_index(drop=True)

In [ ]:
X_test = X_test.reset_index(drop=True)
y_test = y_test.reset_index(drop=True)

In [ ]:
y_test

In [ ]:
from mlxtend.evaluate import bias_variance_decomp
lr = LogisticRegression()


# first calculate all the statistical parameters before pruning
mse_decision_tree, bias_decision_tree, var_decision_tree = bias_variance_decomp(lr,
        df_X_train.to_numpy(), df_y_train.to_numpy(), X_test.to_numpy(), y_test.to_numpy(),
        '0-1_loss', random_seed=123 )

# random_seed : Used to initialize a pseudo-random
# number generator for the bias-variance decomposition
print('Original Bias from un-pruned data ', np.round(bias_decision_tree, 2))
print('Original Variance from un-pruned data ', np.round(var_decision_tree, 2))

In [ ]:
df_X_train.to_numpy()